In [ ]:
Zt_cov = np.zeros((Zts.shape[1],Zts.shape[1],Zts.shape[2]))
x_t = np.zeros((Zts.shape[1],Zts.shape[2]))
for i_t in range(Zts.shape[2]):
    Zt = Zts[:,:,i_t]
    Zt_cov[:,:,i_t] = (Zt.T).dot(Zt)
    if i_t<Zts.shape[2]-1:
        x_t[:,i_t] = (Zt.T).dot(rs[:,i_t+1])

In [ ]:
np.random.seed(4134132)
K = 5
L = Zts.shape[1]
N = Zts.shape[0]
# initialization 
Gamma_b_0 = np.random.rand(L,K)
Gamma_b_1 = np.array(Gamma_b_0, copy=True)
F_0 = np.random.rand(K,T)
F_1 = np.array(F_0, copy=True)
iter = 0
tol = 1e-6
start_time = time.time()
max_iter = 1001 
max_abs_delta_F = np.zeros(max_iter)
max_abs_delta_Gamma_b = np.zeros(max_iter)
while iter < max_iter:
    # update F
    for i_t in range(T-1): 
        #Zt = Zts[:,:,i_t]
        mat = Gamma_b_0.T.dot(Zt_cov[:,:,i_t]).dot(Gamma_b_0)
        F_1[:,i_t+1] = np.linalg.inv(mat).dot(Gamma_b_0.T).dot(x_t[:,i_t]) 
    # update Gamma_b
    mat_1 = np.zeros((L*K,L*K))
    mat_2 = np.zeros(L*K)
    for i_t in range(T-1):
        #Zt = Zts[:,:,i_t]
        #mat_0 = np.kron(Zt, F_1[:,i_t+1].T)
        cur_F = F_1[:,i_t+1].reshape(len(cur_F),1)
        mat_1 += np.kron(Zt_cov[:,:,i_t], (cur_F.dot(cur_F.T)) ) # mat_0.T.dot(mat_0)
        mat_2 += np.kron(x_t[:,i_t], cur_F.flatten()) #mat_0.T.dot(rs[:,i_t+1])
    Gamma_b_1 = np.linalg.inv(mat_1).dot(mat_2).reshape(L,K)
    # normalization 
    U,S,V = np.linalg.svd(Gamma_b_1, full_matrices=False)
    Gamma_b_1 = Gamma_b_1.dot(V.T).dot(np.diag(1/S))

    max_abs_delta_F[iter] = np.sum(np.abs(F_0-F_1)) / (T * K)
    max_abs_delta_Gamma_b[iter] = np.sum(np.abs(Gamma_b_0-Gamma_b_1)) / (L * K)
    # max_abs_delta_F[iter] = np.amax(np.abs(F_0-F_1))
    # max_abs_delta_Gamma_b[iter] = np.amax(np.abs(Gamma_b_0-Gamma_b_1))
    
    if max_abs_delta_F[iter] <= tol and max_abs_delta_Gamma_b[iter] <= tol:  
        print('iter', iter, '\tmax_abs_delta_F', max_abs_delta_F[iter], 
              '\tmax_abs_delta_Gamma_b', max_abs_delta_Gamma_b[iter])
        break
    if not iter % 100: 
        print('iter', iter, '\tmax_abs_delta_F', max_abs_delta_F[iter], 
              '\tmax_abs_delta_Gamma_b', max_abs_delta_Gamma_b[iter])
    Gamma_b_0 = np.array(Gamma_b_1, copy=True)
    F_0 = np.array(F_1, copy=True)

    iter += 1 
end_time = time.time()
print("Elapsed time", end_time - start_time)